In [3]:
H = 128
W = 128
Bin = 32
CR = 64
# Compression Ratio = 64 , Binariser filter = 32
# Compression Ratio = 32 , Binariser filter = 64
# Compression Ratio = 128 , Binariser filter = 16

In [2]:
import cv2 as cv # opencv
import tensorflow as tf #tensorflow
import glob # to read files
import os
import numpy as np
import matplotlib.pyplot as plt
import concurrent.futures

C:\Users\User\Anaconda3\envs\thesis\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [7]:
# #Ignore if already resized
# #Resizing to H X W

# from PIL import Image
# import os, sys
# path = 'E:/ChestXray-NIHCC/images/'
# dirs = os.listdir( path )

# def resize():
#     for item in dirs:
#         if os.path.isfile(path+item):
#             im = cv.imread(path+item)
#             f, e = os.path.splitext(path+item)
#             img = cv.resize(im, (H, W), interpolation = cv.INTER_AREA)
#             cv.imwrite(f+'rs.png', img)

# resize()

In [3]:
dirImage = 'E:/thesis final/Data128'

# filenames = []
filenames = glob.glob(dirImage + "/*" + ".png")

print(np.shape(filenames))
print(filenames[2])

(24999,)
E:/thesis final/Data128\00000001_002rs.png


In [4]:
images = []
for file in filenames:
    img = np.asarray(cv.imread(file, 0))
    
    ######## এইখানে কাটাকুটি করতে হবে কিছু #############
    
    
    images.append(img)
images = np.asarray(images)
print(np.shape(images))

total = len(filenames)
images = np.reshape(images, (total, H, W, 1))
print(np.shape(images))

(24999, 128, 128)
(24999, 128, 128, 1)


In [ ]:
print(np.shape(images[0]))
# cv.imshow('img', images[2002])
# cv.waitKey(0)
# cv.destroyAllWindows()

(128, 128, 1)


In [5]:
print(np.shape(images))

(24999, 128, 128, 1)


In [6]:
tsize = 10000
train_images = images[ :tsize, :, :] 
test_images = images[ tsize: tsize+500, :, :]
valid_images = images[ tsize+500:tsize+1500 , :, :]

In [5]:
# from sklearn import model_selection
# seed = 7
# test_size= 0.1
# X_train, X_test, Y_train, Y_test = model_selection.train_test_split(train_images, train_images, test_size=test_size, random_state=seed)

In [7]:
print(np.shape(train_images))
print(np.shape(test_images))
print(np.shape(valid_images))

(10000, 128, 128, 1)
(500, 128, 128, 1)
(1000, 128, 128, 1)


In [8]:
from keras.utils.np_utils import to_categorical
from __future__ import division


x_train = train_images/255
x_test = test_images/255
x_valid = valid_images/255



y_train = x_train
y_test= x_test
y_valid = x_valid



Using TensorFlow backend.


In [9]:
print(np.shape(x_train), np.shape(y_train))
print(np.shape(x_valid), np.shape(y_valid))
print(np.shape(x_test), np.shape(y_test))

(10000, 128, 128, 1) (10000, 128, 128, 1)
(1000, 128, 128, 1) (1000, 128, 128, 1)
(500, 128, 128, 1) (500, 128, 128, 1)


In [1]:
import keras
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Reshape, Lambda, Input, concatenate
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
from keras.models import Sequential
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from keras import backend as K
from keras.models import load_model
import h5py
from keras.layers import average

C:\Users\User\Anaconda3\envs\thesis\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [78]:
# K.set_floatx('float32')
# K.floatx()

'float32'

In [4]:
##Encoder##

input_img = Input(shape=(H, W, 1))

e1 = Conv2D(64, kernel_size=(3, 3), strides=(2,2), activation='relu', padding= 'same')(input_img)

e2 = MaxPooling2D(pool_size=(2, 2))(input_img)
e2 = Conv2D(64, kernel_size=(1, 1), strides=(1,1), activation='relu', padding= 'same')(e2)

e1 = average([e1, e2])

e1 = Reshape((-1, 64, 64, 64))(e1)    #change this for different size of images

elstm1 = ConvLSTM2D(256, (3, 3), input_shape= (None, 64,64, 64), padding='same', return_sequences=True, strides=(2,2))(e1)

elstm2 = ConvLSTM2D(512, (3, 3), padding='same', return_sequences=True, strides=(2,2))(elstm1)


elstm3 = ConvLSTM2D(512, (3, 3), padding='same', return_sequences=False, strides=(2,2))(elstm2)

#model.add(BatchNormalization(axis=-1))

In [5]:
############# binariser ######################################
b = Conv2D(Bin, kernel_size=(1, 1), strides=(1,1), activation='relu', padding= 'valid')(elstm3)

def cast16(z):
    return K.cast(z, dtype='float16')

xx = Lambda(cast16)(b)

def cast32(z):
    return K.cast(z, dtype='float32')

yy = Lambda(cast32)(xx)

In [6]:
############# decoder ##############################


d1 = Conv2D(512, kernel_size=(1, 1), strides=(1,1), activation='relu', padding= 'valid')(yy)

d1 = Reshape((-1, 8, 8, 512))(d1)    #change this for different size of images


dlstm1 = ConvLSTM2D(512, (2, 2), input_shape=(None, 1, 8, 8, 512), padding='same', return_sequences=False, strides=(1,1))(d1)
dlstm1_ = ConvLSTM2D(512, (1, 1), input_shape=(None, 1, 8, 8, 512), padding='same', return_sequences=False, strides=(1,1))(d1)

#combine
comb1 = average([dlstm1, dlstm1_])

comb1 = Reshape((-1, 16, 16, 128))(comb1)


dlstm2 = ConvLSTM2D(filters=512, kernel_size=(3, 3), padding='same', return_sequences=True, strides=(1,1))(comb1)
dlstm2_ = ConvLSTM2D(filters=512, kernel_size=(1, 1), padding='same', return_sequences=True, strides=(1,1))(comb1)

#combine
comb2 = average([dlstm2, dlstm2_])



comb2 = Reshape((-1, 32, 32, 128))(comb2)


dlstm3 = ConvLSTM2D(filters=256, kernel_size=(2, 2), padding='same', return_sequences=True, strides=(1,1))(comb2)
dlstm3_ = ConvLSTM2D(filters=256, kernel_size=(1, 1), padding='same', return_sequences=True, strides=(1,1))(comb2)

#combine
comb3 = average([dlstm3, dlstm3_])

comb3 = Reshape((-1, 64, 64, 64))(comb3)    


dlstm4 = ConvLSTM2D(filters=128, kernel_size=(2, 2), padding='same', return_sequences=True, strides=(1,1))(comb3)
dlstm4_ = ConvLSTM2D(filters=128, kernel_size=(1, 1), padding='same', return_sequences=True, strides=(1,1))(comb3)

#combine
comb4 = average([dlstm4, dlstm4_])

comb4 = Reshape((H, W, 32))(comb4)


d2 = Conv2D(1, (1, 1), strides=(1,1), input_shape=(None, H, W, 32), activation='relu', padding= 'valid')(comb4)


In [7]:
model = Model(inputs = input_img, outputs = d2)

In [8]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 1)  0                                            
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 64, 64, 1)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 64, 64)   640         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 64, 64, 64)   128         max_pooling2d_1[0][0]            
__________________________________________________________________________________________________
average_1 

In [25]:
# model.layers[5].input.shape

# int_model = Model(inputs = model.input, outputs = model.layers[5].output)
# int_output = int_model.predict(x_test)

In [26]:
# with h5py.File("int_out.hdf5", "w") as f:
#     dset = f.create_dataset("int_out", data = int_output)
#     #, compression="gzip"

In [27]:
# with h5py.File("int_out.hdf5", "r") as f:
#     f['int_out'].shape

In [17]:
weight_path="{}_weights.best.hdf5".format('CR'+ str(CR) +'addModB')


checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)

early = EarlyStopping(monitor= "val_loss", 
                      mode= "min", 
                      patience= 25)
callbacks_list = [checkpoint, early]


In [16]:
model.load_weights(weight_path)

In [19]:
# #Not needed now

# def custom(y_true, y_pred):
#     return K.mean(K.abs(y_pred - y_true)**3, axis=-1)

In [16]:
b_sz = 10
epo = 100

opt = Adam(lr=0.0001)
model.compile(optimizer=opt, loss= 'mae', metrics=['accuracy'])

# for i in range(5):
#     model.fit(x_train[:5000], y_train[:5000], epochs = epo, batch_size= b_sz, callbacks = callbacks_list, validation_data = (x_valid, y_valid))
#     model.load_weights(weight_path)

In [18]:
# load the best weights
model.load_weights(weight_path)

#model.save('compressV2.h5')  # creates a HDF5 file 'compressV2.h5'
#del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
#model = load_model('compressV2.h5')

OSError: Unable to open file (unable to open file: name = 'CR64addModB_weights.best.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [10]:
# model = load_model('compressV2.h5')

In [19]:
#Evaluating the model on test data
model.evaluate(x_test, y_test, batch_size = 10)

500/500 [==============================] - 6s 11ms/step


[0.008988173017278313, 0.0129030762554612]

In [20]:
#model.load_weights(weight_path)

#Predicting the test data
outputs = model.predict(x_test)

In [24]:
for i in range (0, outputs.shape[0]):
    cv.imwrite('temp/'+str(i)+'res.png', outputs[i]*255)
    cv.imwrite('temp/'+str(i)+'org.png', test_images[i])